### Summarization patterns with langchain library
Adapted from langchain's documentation at https://python.langchain.com/en/latest/modules/chains/index_examples/summarize.html

Do pip install ```pip install langchain``` and fill in your information below

In [1]:
from azure.identity import InteractiveBrowserCredential
import os

import openai
interactive_credential = InteractiveBrowserCredential() # add tenant_id="YOUR_TENANT_ID" if you have more than 1 tenants
token = interactive_credential.get_token("https://cognitiveservices.azure.com/.default")

In [7]:
from langchain.chat_models.azure_openai import AzureChatOpenAI
from langchain.agents.react.base import DocstoreExplorer
GPT_ENGINE = "chatGpt-35-turbo"
os.environ["LANGCHAIN_HANDLER"] = "langchain"
os.environ["OPENAI_API_KEY"] = "" # replace with API key in case you don't use Azure AAD
os.environ["OPENAI_API_TYPE"] = "azure" # replace with "azure" in case you don't use Azure AAD
service_name = "kdopenai-sc"
os.environ["OPENAI_API_BASE"] = f"https://{service_name}.openai.azure.com/"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
llm= AzureChatOpenAI(deployment_name=GPT_ENGINE,temperature =0)

from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
text_splitter = CharacterTextSplitter()

In [4]:
with open("state_of_the_union.txt", encoding='utf-8') as f:
    state_of_the_union = f.read()
texts = text_splitter.split_text(state_of_the_union)

In [5]:
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain

docs = [Document(page_content=t) for t in texts[:3]]

##### With map reduce pattern

In [8]:
chain = load_summarize_chain(llm, chain_type="map_reduce")
chain.run(docs)

"President Biden addressed Congress and the nation, emphasizing the importance of American diplomacy and resolve in standing with Ukraine against Russian aggression. The US is taking robust action to target Russia's economy with sanctions and providing military, economic, and humanitarian assistance to Ukraine. The President also highlighted the success of the American Rescue Plan in providing economic relief and announced the passing of the Bipartisan Infrastructure Law, which will modernize infrastructure and promote environmental justice. He pledged to use taxpayer dollars to support American jobs by buying American products."

##### With map stuff pattern

In [9]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm, chain_type="stuff")

chain.run(docs)


"President Biden addressed Congress and the nation, highlighting the ongoing crisis in Ukraine and the US's response to Russian aggression. He announced economic sanctions against Russia and the mobilization of American ground forces, air squadrons, and ship deployments to protect NATO countries. He also discussed the American Rescue Plan and the Bipartisan Infrastructure Law, emphasizing the need to invest in America's infrastructure to create jobs and compete in the global economy. Biden pledged to use taxpayer dollars to rebuild America by buying American products to support American jobs."

##### With refine pattern

In [10]:
chain = load_summarize_chain(llm, chain_type="refine")

print(chain.run(docs))



President Biden addressed Congress and the nation, highlighting the strength and determination of the Ukrainian people in the face of Russian aggression. He emphasized the importance of American diplomacy and resolve in standing with Ukraine and building a coalition of freedom-loving nations to hold Russia accountable. The President announced the enforcement of powerful economic sanctions and the creation of a task force to go after the crimes of Russian oligarchs. He warned that dictators must pay a price for their aggression, or they will cause more chaos. Additionally, the United States is joining with European allies to seize Russian oligarchs' assets and closing off American airspace to all Russian flights. The US is providing military, economic, and humanitarian assistance to Ukraine, including over $1 billion in direct aid. American forces are mobilized to protect NATO countries, and the US has worked with 30 other countries to release 60 million barrels of oil from reserves a

In [11]:
chain = load_summarize_chain(llm, chain_type="refine", return_intermediate_steps=True)

chain({"input_documents": docs}, return_only_outputs=True)


{'intermediate_steps': ['President Biden addressed Congress and the nation, highlighting the strength and determination of the Ukrainian people in the face of Russian aggression. He emphasized the importance of American diplomacy and resolve in standing with Ukraine and building a coalition of freedom-loving nations to hold Russia accountable. The President announced the enforcement of powerful economic sanctions and the creation of a task force to go after the crimes of Russian oligarchs. He warned that dictators must pay a price for their aggression, or they will cause more chaos.',
  "\n\nPresident Biden addressed Congress and the nation, highlighting the strength and determination of the Ukrainian people in the face of Russian aggression. He emphasized the importance of American diplomacy and resolve in standing with Ukraine and building a coalition of freedom-loving nations to hold Russia accountable. The President announced the enforcement of powerful economic sanctions and the c

##### Customize prompt 

In [12]:
prompt_template = """Write a concise summary of the following:


{text}


CONCISE SUMMARY:"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary. Make sure to address the list of problems, list of solutions and any following action"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)
chain = load_summarize_chain(llm, chain_type="refine", return_intermediate_steps=True, question_prompt=PROMPT, refine_prompt=refine_prompt)
chain({"input_documents": docs}, return_only_outputs=True)

{'intermediate_steps': ['President Biden addressed Congress and the nation, highlighting the strength and determination of the Ukrainian people in the face of Russian aggression. He emphasized the importance of American diplomacy and resolve in standing with Ukraine and its allies, and announced the imposition of economic sanctions on Russia, including cutting off access to technology and targeting Russian oligarchs. The U.S. Department of Justice is also assembling a task force to go after the crimes of Russian oligarchs.',
  '\n\nPresident Biden addressed Congress and the nation, highlighting the strength and determination of the Ukrainian people in the face of Russian aggression. He emphasized the importance of American diplomacy and resolve in standing with Ukraine and its allies, and announced the imposition of economic sanctions on Russia, including cutting off access to technology and targeting Russian oligarchs. The U.S. Department of Justice is also assembling a task force to 